In [ ]:
# Intel realsense test
import os
import cv2
import pyrealsense2 as rs
import numpy as np
from fastdlo.core import Pipeline

#########################
# Set up the parameters #
#########################
'''
# YOU MIGHT NEED TO TOUCH THIS PARAMETERS!
'''
IMG_W = 640 # image size
IMG_H = 480
R = np.array([[0,1],[1,0]]) # Rotation matrix for coordinate fitting
Distance_threshold = 60 # distance detection threshold
colorRange = [
    ((0, 43, 46),(10, 255, 255)),
    ((156, 43, 46),(180, 255, 255))
]




# weighting file name - NO NEED TO TOUCH
ckpt_siam_name = "CP_similarity.pth"
ckpt_seg_name = "CP_segmentation.pth"

# get current run file path - NO NEED TO TOUCH
script_path = os.getcwd()
# get network weights <- need to create folder named 'weights' under main program folder, paste *.ckpt/*.pth files in it
checkpoint_siam = os.path.join(script_path, "weights/" + ckpt_siam_name)
checkpoint_seg = os.path.join(script_path, "weights/" + ckpt_seg_name)
# load FASTDLO algorithm pipeline - NO NEED TO TOUCH
p = Pipeline(checkpoint_siam=checkpoint_siam, checkpoint_seg=checkpoint_seg, img_w=IMG_W, img_h=IMG_H, colorRange=colorRange)

# set up the realsense pipeline & config - NO NEED TO TOUCH
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.color, IMG_W, IMG_H, rs.format.rgb8, 30)
pipeline.start(config)

##############################
# run the detection progress #
##############################
try:
    while True:
        frames = pipeline.wait_for_frames()
        color_frame = frames.get_color_frame()
        if not color_frame:
            continue

        color_image = np.asanyarray(color_frame.get_data())
        # run the FASTDLO, return a array of size (IMG_H, IMG_W, 3)
        img_out, dlo_mask_pointSet, dlo_path = p.run(source_img=color_image, mask_th=127)
        
        for i in range(len(dlo_path)):
            pos = dlo_path[i]
            pos = R.dot(pos) # coordinate btw image and path fitting
            if i != len(dlo_path)-1:
                pos2 = dlo_path[i+1]
                pos2 = R.dot(pos2)
                # detect whether the points in between are too far away, 
                # if not, then connect them with line segment
                if np.sqrt(((pos[0]-pos2[0])**2)+((pos[1]-pos2[1])**2)) < Distance_threshold:
                    cv2.line(color_image, pt1=pos, pt2=pos2, color=(0,255,50))
                else:
                    pass
            else:
                pos2 = None
            # draw the path line marker
            cv2.drawMarker(color_image, tuple(pos), color=(0,255,50), markerType=3, markerSize=7, thickness=1)

        canvas = color_image.copy()
        # show the detected results with origin stream video together with weight 0.5 for each 
        canvas = cv2.addWeighted(canvas, 1.0, img_out, 0.3, 0.0)
        
        cv2.namedWindow("RealSense", cv2.WINDOW_AUTOSIZE) # set up the window to display the results
        cv2.imshow("RealSense", canvas)
        
        key = cv2.waitKey(1)
        if key & 0xFF == ord('q') or key == 27: # press the keyboard 'q' or 'esc' to terminate the thread
            cv2.destroyAllWindows()
            break
finally:
    pipeline.stop()

In [1]:
# image test

import os, cv2
from fastdlo.core import Pipeline
import numpy as np
import matplotlib.pyplot as plt

IMG_PATH = 'test_images/7.jpg'
ckpt_siam_name = "CP_similarity.pth"
ckpt_seg_name = "CP_segmentation.pth"
IMG_W = 640
IMG_H = 480
R = np.array([[0,1],[1,0]])

colorRange = [
    ((0, 43, 46),(10, 255, 255)),
    ((156, 43, 46),(180, 255, 255))
]
activate_interpolation = True

######################
# Visit current path
script_path = os.getcwd()
# get network weights <- need to create weights folder under main program folder, save *.ckpt/*.pth files in it
checkpoint_siam = os.path.join(script_path, "weights/" + ckpt_siam_name)
checkpoint_seg = os.path.join(script_path, "weights/" + ckpt_seg_name)


p = Pipeline(checkpoint_siam=checkpoint_siam, checkpoint_seg=checkpoint_seg, 
img_w=IMG_W, img_h=IMG_H, colorRange=colorRange, is_interpolation=activate_interpolation)

# COLOR

# convert image into BGR Format, IMG_PATH also include image file name *.jpg
source_img = cv2.imread(IMG_PATH, cv2.IMREAD_COLOR)
source_img = cv2.resize(source_img, (IMG_W, IMG_H))

# run the semantic segmentation using pipeline
img_out, splines, path_final = p.run(source_img=source_img, mask_th=70)
'''
spline coordinate frame set up at top left of the image, 
the x-direction points down, the y-direction points right 
'''

for i in range(len(path_final)):
    pos = path_final[i]
    pos = R.dot(pos) # coordinate btw image and path fitting
    if i != len(path_final)-1:
        pos2 = path_final[i+1]
        pos2 = R.dot(pos2)
        # detect whether the points in between far away, if not, then connect them with line segment
        if np.sqrt(((pos[0]-pos2[0])**2)+((pos[1]-pos2[1])**2)) < 60:
            cv2.line(source_img, pt1=pos, pt2=pos2, color=(255,50,0), thickness=1)
        else:
            pass
    else:
        pos2 = None
    cv2.drawMarker(source_img, tuple(pos), color=(255,50,0), markerType=3, markerSize=7, thickness=1)
print(path_final)
canvas = source_img.copy()
canvas = cv2.addWeighted(canvas, 1.0, img_out, 0.2, 0.0)
while True:
    cv2.imshow("output", canvas)
    key = cv2.waitKey(1)
    if key & 0xFF == ord('q') or key == 27: # press the keyboard 'q' or 'esc' to terminate the thread
        cv2.destroyAllWindows()
        break



/home/yuan/Desktop/Master Thesis/src/cv_related_src/FASTDLO_IntelRealSense/fastdlo/siam_net/nn_dataset.py:22: RuntimeWarning: divide by zero encountered in double_scalars
  angle = math.atan(dir[1]/dir[0])


[array([  7, 330]), array([ 14, 336]), array([ 21, 342]), array([ 28, 348]), array([ 35, 353]), array([ 42, 357]), array([ 49, 362]), array([ 56, 367]), array([ 63, 371]), array([ 70, 375]), array([ 77, 379]), array([ 84, 383]), array([ 91, 387]), array([ 98, 390]), array([105, 394]), array([112, 398]), array([119, 402]), array([126, 406]), array([133, 408]), array([140, 412]), array([147, 415]), array([154, 419]), array([161, 421]), array([168, 424]), array([175, 427]), array([182, 429]), array([189, 432]), array([196, 434]), array([203, 437]), array([210, 439]), array([217, 441]), array([224, 443]), array([231, 444]), array([238, 446]), array([245, 448]), array([252, 450]), array([259, 452]), array([266, 453]), array([273, 455]), array([280, 456]), array([287, 457]), array([315, 460]), array([322, 459]), array([329, 459]), array([336, 459]), array([343, 459]), array([350, 458]), array([357, 457]), array([364, 457]), array([371, 456]), array([378, 456]), array([385, 456]), array([392,

QObject::moveToThread: Current thread (0x19db3190) is not the object's thread (0x7ffc3ee0).
Cannot move to target thread (0x19db3190)

QObject::moveToThread: Current thread (0x19db3190) is not the object's thread (0x7ffc3ee0).
Cannot move to target thread (0x19db3190)

QObject::moveToThread: Current thread (0x19db3190) is not the object's thread (0x7ffc3ee0).
Cannot move to target thread (0x19db3190)

QObject::moveToThread: Current thread (0x19db3190) is not the object's thread (0x7ffc3ee0).
Cannot move to target thread (0x19db3190)

QObject::moveToThread: Current thread (0x19db3190) is not the object's thread (0x7ffc3ee0).
Cannot move to target thread (0x19db3190)

QObject::moveToThread: Current thread (0x19db3190) is not the object's thread (0x7ffc3ee0).
Cannot move to target thread (0x19db3190)

QObject::moveToThread: Current thread (0x19db3190) is not the object's thread (0x7ffc3ee0).
Cannot move to target thread (0x19db3190)

QObject::moveToThread: Current thread (0x19db3190) is n

In [2]:
import numpy as np
from numpy import array

data = [array([  7, 330]), array([ 14, 336]), array([ 21, 342]), array([ 28, 348]), array([ 35, 353]), 
array([ 42, 357]), array([ 49, 362]), array([ 56, 367]), array([ 63, 371]), array([ 70, 375]), 
array([ 77, 379]), array([ 84, 383]), array([ 91, 387]), array([ 98, 390]), array([105, 394]), 
array([112, 398]), array([119, 402]), array([126, 406]), array([133, 408]), array([140, 412]), 
array([147, 415]), array([154, 419]), array([161, 421]), array([168, 424]), array([175, 427]), 
array([182, 429]), array([189, 432]), array([196, 434]), array([203, 437]), array([210, 439]), 
array([217, 441]), array([224, 443]), array([231, 444]), array([238, 446]), array([245, 448]), 
array([252, 450]), array([259, 452]), array([266, 453]), array([273, 455]), array([280, 456]), 
array([287, 457]), array([315, 460]), array([322, 459]), array([329, 459]), array([336, 459]), 
array([343, 459]), array([350, 458]), array([357, 457]), array([364, 457]), array([371, 456]), 
array([378, 456]), array([385, 456]), array([392, 455]), array([399, 454]), array([406, 453]), 
array([413, 453]), array([420, 452]), array([427, 451]), array([434, 450]), array([441, 449]), 
array([448, 448]), array([455, 447]), array([462, 446]), array([470, 445]), array([439, 115]), 
array([443, 108]), array([448, 100]), array([451,  93]), array([456,  85]), array([459,  78]), 
array([464,  70]), array([468,  63]), array([472,  55]), array([224,   7]), array([228,  14]), 
array([234,  21]), array([239,  28]), array([244,  35]), array([247,  42]), array([251,  49]), 
array([255,  56]), array([258,  64]), array([262,  71]), array([264,  78]), array([267,  85]), 
array([269,  92]), array([272,  99]), array([274, 106]), array([276, 113]), array([278, 121]), 
array([280, 128]), array([282, 135]), array([282, 142]), array([284, 149]), array([285, 156]), 
array([287, 163]), array([289, 171]), array([289, 178]), array([290, 185]), array([290, 192]), 
array([291, 199]), array([291, 206]), array([292, 213]), array([292, 220]), array([293, 228]), 
array([292, 235]), array([292, 242]), array([293, 249]), array([294, 256]), array([296, 263]), 
array([297, 270]), array([297, 278]), array([291, 353]), array([291, 360]), array([292, 367]), 
array([292, 374]), array([292, 381]), array([292, 388]), array([292, 395]), array([293, 403]), 
array([293, 410]), array([294, 417]), array([295, 424]), array([297, 431]), array([298, 438]), 
array([298, 446]), array([307, 471]), array([309, 478]), array([310, 485]), array([313, 492]), 
array([315, 499]), array([317, 506]), array([320, 513]), array([323, 521]), array([326, 528]), 
array([330, 535]), array([334, 542]), array([338, 549]), array([342, 556]), array([348, 564]), 
array([353, 571]), array([358, 578]), array([363, 585]), array([370, 591]), array([375, 598]), 
array([383, 606]), array([390, 612]), array([397, 618]), array([404, 623]), array([411, 628]), 
array([418, 631]), array([426, 634])]

dis_list = []

for i in range(len(data)):
    pos = data[i]
    pos = R.dot(pos) # coordinate btw image and path fitting
    if i != len(data)-1:
        pos2 = data[i+1]
        pos2 = R.dot(pos2)
        # detect whether the points in between far away, if not, then connect them with line segment
        length = np.sqrt(((pos[0]-pos2[0])**2)+((pos[1]-pos2[1])**2))
        dis_list.append(length)

dis_list

[9.219544457292887,
 9.219544457292887,
 9.219544457292887,
 8.602325267042627,
 8.06225774829855,
 8.602325267042627,
 8.602325267042627,
 8.06225774829855,
 8.06225774829855,
 8.06225774829855,
 8.06225774829855,
 8.06225774829855,
 7.615773105863909,
 8.06225774829855,
 8.06225774829855,
 8.06225774829855,
 8.06225774829855,
 7.280109889280518,
 8.06225774829855,
 7.615773105863909,
 8.06225774829855,
 7.280109889280518,
 7.615773105863909,
 7.615773105863909,
 7.280109889280518,
 7.615773105863909,
 7.280109889280518,
 7.615773105863909,
 7.280109889280518,
 7.280109889280518,
 7.280109889280518,
 7.0710678118654755,
 7.280109889280518,
 7.280109889280518,
 7.280109889280518,
 7.280109889280518,
 7.0710678118654755,
 7.280109889280518,
 7.0710678118654755,
 7.0710678118654755,
 28.160255680657446,
 7.0710678118654755,
 7.0,
 7.0,
 7.0,
 7.0710678118654755,
 7.0710678118654755,
 7.0,
 7.0710678118654755,
 7.0,
 7.0,
 7.0710678118654755,
 7.0710678118654755,
 7.0710678118654755,
 7.0

In [ ]:
# laptop camera test

import cv2
import numpy as np
from fastdlo.core import Pipeline

colorRange = [
    ((0, 43, 46),(10, 255, 255)),
    ((156, 43, 46),(180, 255, 255))
]
IMG_W = 640
IMG_H = 480
R = np.array([[0,1],[1,0]])
Distance_threshold = 30 # distance detection threshold
ckpt_siam_name = "CP_similarity.pth"
ckpt_seg_name = "CP_segmentation.pth"
script_path = os.getcwd()
# get network weights <- need to create weights folder under main program folder, save *.ckpt/*.pth files in it
checkpoint_siam = os.path.join(script_path, "weights/" + ckpt_siam_name)
checkpoint_seg = os.path.join(script_path, "weights/" + ckpt_seg_name)


p = Pipeline(checkpoint_siam=checkpoint_siam, checkpoint_seg=checkpoint_seg, 
img_w=IMG_W, img_h=IMG_H, colorRange=colorRange, is_interpolation=True)

cap = cv2.VideoCapture(0)

while True:
    ret,frame =cap.read()
    # ret will return a true value if the frame exists otherwise False
    img_out, dlo_mask_pointSet, dlo_path = p.run(source_img=frame, mask_th=77)
    
    for i in range(len(dlo_path)):
        pos = dlo_path[i]
        pos = R.dot(pos) # coordinate btw image and path fitting
        if i != len(dlo_path)-1:
            pos2 = dlo_path[i+1]
            pos2 = R.dot(pos2)
            # detect whether the points in between are too far away, 
            # if not, then connect them with line segment
            if np.sqrt(((pos[0]-pos2[0])**2)+((pos[1]-pos2[1])**2)) < Distance_threshold:
                cv2.line(frame, pt1=pos, pt2=pos2, color=(0,255,50))
            else:
                pass
        else:
            pos2 = None
        # draw the path line marker
        cv2.drawMarker(frame, tuple(pos), color=(0,255,50), markerType=3, markerSize=7, thickness=1)

    canvas = frame.copy()
    # show the detected results with origin stream video together with weight 0.5 for each 
    canvas = cv2.addWeighted(canvas, 1.0, img_out, 0.3, 0.0)
    
    cv2.namedWindow("laptop camera", cv2.WINDOW_AUTOSIZE) # set up the window to display the results
    cv2.imshow("laptop camera", canvas)
        
 

 
    if cv2.waitKey(1)==27:
        break
    # this function will be triggered when the ESC key is pressed
    # and the while loop will terminate and so will the program
cap.release()
 
cv2.destroyAllWindows()

In [ ]:
# interpolation
import matplotlib.pyplot as plt
import numpy as np

x = [-3, -2, -1, 0, 1, 2, 3, 4, 5, 6, 7]
y = [14, 7, 5, 1, 2, 6, 7, 8, 13, 20, 21]


def lagrange(x, y, num_points, x_test):
    # 所有的基函数值，每个元素代表一个基函数的值
    l = np.zeros(shape=(num_points, ))

    # 计算第k个基函数的值
    for k in range(num_points):
        # 乘法时必须先有一个值
        # 由于l[k]肯定会被至少乘n次，所以可以取1
        l[k] = 1
        # 计算第k个基函数中第k_个项（每一项：分子除以分母）
        for k_ in range(num_points):
            # 这里没搞清楚，书中公式上没有对k=k_时，即分母为0进行说明
            # 有些资料上显示k是不等于k_的
            if k != k_:
                # 基函数需要通过连乘得到
                l[k] = l[k]*(x_test-x[k_])/(x[k]-x[k_])
            else:
                pass 
    # 计算当前需要预测的x_test对应的y_test值        
    L = 0
    for i in range(num_points):
        # 求所有基函数值的和
        L += y[i]*l[i]
    return L

fig, ax = plt.subplots()


In [ ]:
# interpolation candidate 1
import numpy as np
N = 21
x = np.linspace(0, 10, 2)
y = [3.9, 4.4]

# fit a linear curve an estimate its y-values and their error.
a, b = np.polyfit(x, y, deg=1)
y_est = a * x + b
y_err = x.std() * np.sqrt(1/len(x) +
                          (x - x.mean())**2 / np.sum((x - x.mean())**2))

fig, ax = plt.subplots()
ax.plot(x, y_est, '-')
ax.fill_between(x, y_est - y_err, y_est + y_err, alpha=0.2)
ax.plot(x, y, 'o', color='tab:brown')

In [ ]:
# interpolation candidate 2
import matplotlib.pyplot as plt
import numpy as np
from scipy.interpolate import UnivariateSpline

rng = np.random.default_rng()
x = np.linspace(-3, 3, 50)
y = np.exp(-x**2) + 0.1 * rng.standard_normal(50)
plt.plot(x, y, 'ro', ms=5)

spl = UnivariateSpline(x, y)
xs = np.linspace(-3, 3, 1000)
plt.plot(xs, spl(xs), 'g', lw=3)

spl.set_smoothing_factor(0.5)
plt.plot(xs, spl(xs), 'b', lw=3)
plt.show()
print(x)

In [ ]:
import numpy as np
from numpy import array

data = [array([  7, 330]), array([ 14, 336]), array([ 21, 342]), array([ 28, 348]), 
array([ 35, 353]), array([ 42, 357]), array([ 49, 362]), array([ 56, 367]), array([ 63, 371]), 
array([ 70, 375]), array([ 77, 379]), array([ 84, 383]), array([ 91, 387]), array([ 98, 390]), 
array([105, 394]), array([112, 398]), array([119, 402]), array([126, 406]), array([133, 408]), 
array([140, 412]), array([147, 415]), array([154, 419]), array([161, 421]), array([168, 424]), 
array([175, 427]), array([182, 429]), array([189, 432]), array([196, 434]), array([203, 437]), 
array([210, 439]), array([217, 441]), array([224, 443]), array([231, 444]), array([238, 446]), 
array([245, 448]), array([252, 450]), array([259, 452]), array([266, 453]), array([273, 455]), 
array([280, 456]), array([287, 457]), array([315, 460]), array([322, 459]), array([329, 459]), 
array([336, 459]), array([343, 459]), array([350, 458]), array([357, 457]), array([364, 457]), 
array([371, 456]), array([378, 456]), array([385, 456]), array([392, 455]), array([399, 454]), 
array([406, 453]), array([413, 453]), array([420, 452]), array([427, 451]), array([434, 450]), 
array([441, 449]), array([448, 448]), array([455, 447]), array([462, 446]), array([470, 445]), 
array([224,   7]), array([228,  14]), array([234,  21]), array([239,  28]), array([244,  35]), 
array([247,  42]), array([251,  49]), array([255,  56]), array([258,  64]), array([262,  71]), 
array([264,  78]), array([267,  85]), array([269,  92]), array([272,  99]), array([274, 106]), 
array([276, 113]), array([278, 121]), array([280, 128]), array([282, 135]), array([282, 142]), 
array([284, 149]), array([285, 156]), array([287, 163]), array([289, 171]), array([289, 178]), 
array([290, 185]), array([290, 192]), array([291, 199]), array([291, 206]), array([292, 213]), 
array([292, 220]), array([293, 228]), array([292, 235]), array([292, 242]), array([293, 249]), 
array([294, 256]), array([296, 263]), array([297, 270]), array([297, 278]), array([291, 353]), 
array([291, 360]), array([292, 367]), array([292, 374]), array([292, 381]), array([292, 388]), 
array([292, 395]), array([293, 403]), array([293, 410]), array([294, 417]), array([295, 424]), 
array([297, 431]), array([298, 438]), array([298, 446]), array([307, 471]), array([309, 478]), 
array([310, 485]), array([313, 492]), array([315, 499]), array([317, 506]), array([320, 513]), 
array([323, 521]), array([326, 528]), array([330, 535]), array([334, 542]), array([338, 549]), 
array([342, 556]), array([348, 564]), array([353, 571]), array([358, 578]), array([363, 585]), 
array([370, 591]), array([375, 598]), array([383, 606]), array([390, 612]), array([397, 618]), 
array([404, 623]), array([411, 628]), array([418, 631]), array([426, 634]), array([439, 115]), 
array([443, 108]), array([448, 100]), array([451,  93]), array([456,  85]), array([459,  78]), 
array([464,  70]), array([468,  63]), array([472,  55])]

for i in range(len(data)):
    pos = data[i]
    pos = R.dot(pos) # coordinate btw image and path fitting
    if i != len(data)-1:
        pos2 = data[i+1]
        pos2 = R.dot(pos2)
        # detect whether the points in between far away, if not, then connect them with line segment
        length = np.sqrt(((pos[0]-pos2[0])**2)+((pos[1]-pos2[1])**2))
        # Notation the closest point distance is about 10, so the interpolated points vancancy should be at least 20
        if length >= 20:
            # calculate the number of points should be interpolated
            num_interpolation = int(length / 10 - 1)
            assert num_interpolation > 0
            
            x = [pos[0], pos2[0]]
            y = [pos[1], pos2[1]]
            # find linear fitting curve
            a, b = np.polyfit(x, y, deg=1)
            
            for idx in range(num_interpolation):
                # calculate interpolation points
                x_interpolate = pos[0] + (pos2[0] - pos[0])*((num_interpolation - idx)/(num_interpolation+1))
                y_interpolate = a * x_interpolate + b
                data.insert(i+1,np.array([int(x_interpolate), int(y_interpolate)]))
            
        else:
            pass
    else:
        pos2 = None
data       

In [ ]:
from numpy import array

path_final = {0: {'points': [array([34,  6]), array([36, 12]), array([39, 19]), array([42, 26]), array([46, 33]), 
array([50, 40]), array([53, 47]), array([57, 54]), array([61, 61]), array([64, 68]), array([68, 75]), 
array([73, 82]), array([76, 89]), array([110, 129]), array([112, 135]), array([117, 142]), array([123, 149]),
 array([128, 156]), array([134, 163]), array([140, 170]), array([146, 177]), array([152, 184]), 
 array([158, 191]), array([164, 198]), array([170, 205]), array([175, 212]), array([182, 219]), 
 array([188, 226]), array([194, 233]), array([199, 239]), array([206, 246]), array([212, 253]), 
 array([218, 260]), array([224, 266]), array([231, 272]), array([238, 278]), array([244, 284]), 
 array([251, 290]), array([258, 296]), array([265, 301]), array([272, 306]), array([279, 312]), 
 array([286, 317]), array([293, 321]), array([300, 326]), array([307, 330]), array([314, 333]), 
 array([321, 336]), array([328, 339]), array([335, 340]), array([342, 343]), array([349, 344]), 
 array([356, 345]), array([363, 346]), array([370, 346]), array([377, 346]), array([384, 346]), 
 array([391, 347]), array([398, 347]), array([405, 348]), array([410, 344]), array([413, 337]), 
 array([411, 330]), array([406, 324]), array([399, 319]), array([392, 314]), array([385, 309]), 
 array([378, 304]), array([371, 299]), array([364, 294]), array([357, 289]), array([350, 284]), 
 array([343, 278]), array([336, 274]), array([329, 267]), array([322, 262]), array([315, 256]), 
 array([308, 250]), array([301, 244]), array([294, 238]), array([287, 231]), array([280, 226]), 
 array([273, 220]), array([266, 214]), array([259, 208]), array([252, 203]), array([245, 197]), 
 array([239, 192]), array([232, 185]), array([225, 180]), array([218, 175]), array([211, 170]), 
 array([204, 165]), array([197, 160]), array([190, 155]), array([183, 151]), array([176, 147]), 
 array([169, 142]), array([162, 138]), array([155, 134]), array([148, 130]), array([141, 126]), 
 array([134, 122]), array([127, 118]), array([120, 117])], 
 'radius': 6.604565, 
 'nodes': [9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, \
    30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, \
 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 
 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, \
 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113]}, \

 1: {'points': [array([ 7, 86]), array([14, 88]), array([21, 89]), array([29, 92]), array([36, 93]), 
 array([43, 95]), array([51, 97]), array([58, 98]), array([ 66, 100])], 'radius': 5.7619967},
 2:{'points':[array([0,0]), array([20,20]), array([30,30])]},
 3:{'points':[array([200, 160]), array([330, 400])]}}

points_path = []

for i, item in enumerate(path_final.values()):
    points_path.append(item['points'])
spline_num = len(points_path)
# if spline_num == 1:
#     return points_path[0]

count = spline_num - 1

# take the first spline
final_path = points_path[0]

while count > 0:
    head = final_path[0]
    tail = final_path[-1]
    map1 = []
    map2 = []
    # calculate the closest distance btw head & tail to the first spline, and find smallest one
    for path in points_path[1:]:
        son_head = path[0]
        son_tail = path[-1]
        length1 = np.sqrt(((head[0]-son_tail[0])**2)+((head[1]-son_tail[1])**2))
        length2 = np.sqrt(((son_head[0]-tail[0])**2)+((son_head[1]-tail[1])**2))
        map1.append(length1)
        map2.append(length2)
    
    map = np.array([map1,map2])
    h,w = map.shape
    pos = map.argmin()
    idx_row, idx_col = pos // w, pos % w
    # find closest distance and its connection position
    if idx_row == 0:
        points_path[idx_col+1].extend(points_path[0])
        final_path = points_path[idx_col+1]
        points_path[0] = final_path
        del points_path[idx_col+1]
    else:
        final_path.extend(points_path[idx_col+1])
        del points_path[idx_col+1]
    points_path[0] = final_path

    count -= 1

final_path


In [ ]:
import numpy as np
ary = np.mat([[ 99.2975327 ],[117.17508268]])
print(np.argmin(ary,axis=1))

In [ ]:
lst = []
len(lst)